# Initializing RAG

This will load the model, vectorstore, and db. If running for the first time, it will download the model and set up vectorstore which will take some time.

In [2]:
from settings import init_rag

init_rag()

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-large-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
INFO:logger:[SUCCESS] RAG system initialized successfully.
